# Week 3 Project

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

 1. Start by creating a new Notebook for this assignment.
 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1588723200000&hmac=JL089sd48RiJrqZ6ldaIAfnU1la-BG2u0JnAsa9hviI" width="400" height="400">

3. To create the above dataframe:

 * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
 * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
 * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
 * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
 

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [1]:
# !pip3 install BeautifulSoup4
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Scrape the content from Wikipedia using BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r  = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

table = soup.find('table', attrs={'class': 'wikitable sortable'})

In [3]:
# Use Pandas built-in method read_html to convert table on a web into a dataframe
neighborhood=pd.read_html(str(table))[0]
neighborhood.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


### Exclude rows with a borough value `Not assigned`.

In [4]:
neighborhood_cleaned = neighborhood[neighborhood.Borough !='Not assigned']
neighborhood_cleaned.head(10)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,Garden District / Ryerson


### Combine rows with same Postal Code in to one cell, seperated with a comma.

In [5]:
neighborhood_combine = neighborhood_cleaned.groupby(['Postal code', 'Borough']).agg(','.join)
neighborhood_combine.reset_index(inplace = True)
neighborhood_combine.head(10)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
unassigned_neighs = neighborhood_combine['Neighborhood'] == 'Not assigned'
neighborhood_combine.loc[unassigned_neighs,'Neighborhood'] = neighborhood_combine.loc[unassigned_neighs,'Borough']
neighborhood_combine.loc[unassigned_neighs]

,Postal code,Borough,Neighborhood


### Assumptions

* 1. The data on Wikipedia page has been updated - it uses "/" to seperate the data in column `Neighborhood` instead of a ",".
* 2. There is not any rows has a borough with "Not assigned" in column Neighborhood.

### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
neighborhood_combine.shape

(103, 3)

In [8]:
print ("The dataframe has",neighborhood_combine.shape[0],"rows and",neighborhood_combine.shape[1],"columns")

The dataframe has 103 rows and 3 columns
